In [ ]:
client_name = "usdf_efd"  # name of the client. Either "usdf_efd" or "summit_efd"
dayObs = "20250425"  # format YYYYMMDD, e.g. "20250425"
ndays = 15  # number of days to look back, e.g. 15

In [ ]:
## Setup and Imports
import re
import pandas as pd
from astropy.time import Time, TimeDelta

from datetime import datetime, timedelta
from IPython.display import HTML, display

# from lsst_efd_client import EfdClient
from lsst.summit.utils.efdUtils import (
    getEfdData,
    getDayObsEndTime,
    getDayObsStartTime,
    makeEfdClient,
)

import logging

# configure logger to show INFO messages without timestamp and module name
log = logging.getLogger(__name__)
log.handlers.clear()
log.setLevel(logging.INFO)
log.propagate = False

handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
formatter = logging.Formatter("%(levelname)s [%(funcName)s]: %(message)s")
handler.setFormatter(formatter)
log.addHandler(handler)

# M1M3 Following Error Analysis

A concise analysis of M1M3 actuator “Following Error” events that automates data retrieval from the EFD and presents interactive contextual reports with embedded dashboard links.

In [ ]:
def query_m1m3_log_messages(day_obs, ndays, client_name="usdf_efd"):
    """Query M1M3 log messages from the EFD.

    This function retrieves log messages from the M1M3 system for a specified observation day and number of days.

    Parameters
    ----------
    day_obs : str
        The observation day in the format YYYYMMDD.
    ndays : int
        The number of days to look back from the observation day.
    client_name : str
        The name of the EFD client to use for querying data.
        Default is "usdf_efd". It can be either "usdf_efd" or "usdf_efds".

    Returns
    -------
    pd.DataFrame
        A DataFrame containing the log messages and their timestamps.
    """
    possible_clients = ["usdf_efd", "usdf_efds"]
    if client_name not in possible_clients:
        raise ValueError(f"Invalid client name. Choose from {possible_clients}.")
    client = makeEfdClient(client_name)

    start_time = getDayObsStartTime(day_obs) - TimeDelta(
        ndays * 24 * 3600, format="sec"
    )
    end_time = getDayObsEndTime(day_obs)

    log.info(f"Querying MTM1M3.logevent_logMessage from {start_time} to {end_time}")
    try:
        df = getEfdData(
            client=client,
            topic="lsst.sal.MTM1M3.logevent_logMessage",
            columns=["private_sndStamp", "message"],
            begin=Time(start_time),
            end=Time(end_time),
        )
        if df.empty:
            log.warning(f"No data returned between {start_time} and {end_time}")
            return None

        df["private_sndStamp"] = pd.to_datetime(df["private_sndStamp"], unit="s")
        df.rename(columns={"private_sndStamp": "tai_time"}, inplace=True)
        df.index.name = "utc_time"
        return df
    except Exception as e:
        log.error(f"Error fetching M1M3 log messages: {e}")
        return None


# Extract timestamps of 'Following Error' events
def extract_following_error_timestamps(df_messages):
    """Identify timestamps where log messages contain 'Following Error'."""
    return df_messages[df_messages["message"].str.contains("Following Error")].index


# Compute time windows around each error timestamp
def compute_error_time_windows(error_times, tbefore=1, tafter=0.1):
    """Generate start and end times around each error event."""
    return [
        (et - pd.Timedelta(seconds=tbefore), et + pd.Timedelta(seconds=tafter))
        for et in error_times
    ]


# Aggregate messages within each time window and annotate with error context
def aggregate_messages_in_windows(df_messages, time_windows, error_times):
    """Filter and label messages around each 'Following Error' event."""
    filtered = []
    for i, ((start, end), err_ts) in enumerate(zip(time_windows, error_times), 1):
        mask = (df_messages.index >= start) & (df_messages.index <= end)
        tmp = df_messages[mask].copy()
        tmp["range_id"] = i
        tmp["range_start"] = start
        tmp["range_end"] = end
        tmp["following_error_timestamp"] = err_ts
        m = re.search(r"Force Actuator ID (\d+)", df_messages.loc[err_ts, "message"])
        tmp["fa_id"] = int(m.group(1)) if m else None
        filtered.append(tmp)
    return pd.concat(filtered)


# Render 'Following Error' reports as interactive HTML
def render_following_error_reports(df, expanded=False):
    """Display collapsible blocks for each 'Following Error' event with dashboard links."""
    html = """
    <style>
      details {
        margin-bottom: 0.25em;   /* reduced spacing between blocks */
        padding: 0.25em;
      }
      details:hover {
        background-color: #f9f9f9;
        border-radius: 4px;
      }
      summary {
        font-weight: normal;
        cursor: pointer;
      }
      .report-header {
        font-family: Arial, sans-serif;
        color: #2a6f9e;
        margin-bottom: 0.2em;
      }
      .report-instructions {
        font-size: 0.9em;
        color: #555;
        margin-top: 0;
        margin-bottom: 1em;
      }
      hr {
        border: none;
        border-top: 1px solid #ccc;
        margin: 0 0 1em;
      }
    </style>
    <div class="report-header">
      <h2>Following Error Report Summary</h2>
    </div>
    <div class="report-instructions">
      <p>Click on any error entry to expand and view detailed context, including log messages and dashboard links.</p>
    </div>
    <hr>
    """
    for rng, group in df.groupby("range_id"):
        start = group["range_start"].iloc[0]
        end = group["range_end"].iloc[0]
        fa_id = group["fa_id"].iloc[0]
        err_ts = group["following_error_timestamp"].iloc[0]

        summary_text = f"Following Error {rng}: FA ID {fa_id} at {err_ts}"
        details_attr = " open" if expanded else ""

        # build message content with timestamps
        lines = [f"{ts}: {msg}" for ts, msg in zip(group.index, group["message"])]
        content = "<br>".join(lines)

        # compute 2‑min before/after window for dashboard link
        mount_lower_ts = (err_ts - pd.Timedelta(minutes=2)).strftime(
            "%Y-%m-%dT%H:%M:%S.%f"
        )[:-3] + "Z"
        mount_upper_ts = (err_ts + pd.Timedelta(minutes=2)).strftime(
            "%Y-%m-%dT%H:%M:%S.%f"
        )[:-3] + "Z"
        mount_dashboard_url = (
            f"https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/79?"
            f"refresh=Paused&lower={mount_lower_ts}&upper={mount_upper_ts}"
        )
        mount_status_html = (
            f"<div style='margin:5px;'><a href='{mount_dashboard_url}' target='_blank'>"
            "MTMount Status Dashboard</a></div>"
        )

        # compute 1‑min before/after window for Actuator Forces dashboard link
        force_lower_ts = (err_ts - pd.Timedelta(minutes=0.25)).strftime(
            "%Y-%m-%dT%H:%M:%S.%f"
        )[:-3] + "Z"
        force_upper_ts = (err_ts + pd.Timedelta(minutes=0.50)).strftime(
            "%Y-%m-%dT%H:%M:%S.%f"
        )[:-3] + "Z"
        forces_dashboard_url = (
            f"https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/199?"
            f"lower={force_lower_ts}&refresh=Paused"
            f"&tempVars%5Bz_index%5D={fa_id}&tempVars%5By_index%5D={fa_id}"
            f"&tempVars%5Bx_index%5D={fa_id}&tempVars%5Bs_index%5D={fa_id}"
            f"&upper={force_upper_ts}"
        )
        forces_html = (
            f"<div style='margin:5px;'><a href='{forces_dashboard_url}' target='_blank'>"
            "M1M3 Actuator Forces Dashboard</a></div>"
        )

        # compute 1‑min before/after window for TMA Inertial Forces dashboard link
        inertial_lower_ts = (err_ts - pd.Timedelta(minutes=0.5)).strftime(
            "%Y-%m-%dT%H:%M:%S.%f"
        )[:-3] + "Z"
        inertial_upper_ts = (err_ts + pd.Timedelta(minutes=0.5)).strftime(
            "%Y-%m-%dT%H:%M:%S.%f"
        )[:-3] + "Z"
        inertial_dashboard_url = (
            f"https://summit-lsp.lsst.codes/chronograf/sources/1/dashboards/426?"
            f"refresh=Paused&tempVars%5BDownsample%5D=Default"
            f"&tempVars%5BFunction%5D=mean%28%29"
            f"&tempVars%5BForces%5D=all&tempVars%5BHardpoints%5D=all"
            f"&lower={inertial_lower_ts}&upper={inertial_upper_ts}"
        )
        inertial_html = (
            f"<div style='margin:5px;'><a href='{inertial_dashboard_url}' target='_blank'>"
            "M1M3 TMA Inertial Forces - Min Max Dashboard</a></div>"
        )

        html += (
            f"<details{details_attr}>"
            f"<summary>{summary_text}</summary>"
            f"<pre style='margin-left:1em;font-family:monospace;'>"
            f"{content}"
            f"</pre>"
            f"    <pr> &nbsp&nbsp Choronograf Dashboards: <br></pr>\n"
            f"    <br>"
            f"    {forces_html}"
            f"    {inertial_html}"
            f"    {mount_status_html}"
            f"    <br>"
            "</details>"
        )

    display(HTML(html))

# Display Following Error Events
---

In [ ]:
# Query, filter and display 'Following Error' events
df_messages = query_m1m3_log_messages(dayObs, ndays=ndays, client_name=client_name)

# Extract timestamps of 'Following Error' events
error_times = extract_following_error_timestamps(df_messages)

# Compute time windows around each error timestamp
time_windows = compute_error_time_windows(error_times, tbefore=1, tafter=0.1)

# Filter messages within the time windows and annotate with error context
filtered_messages = aggregate_messages_in_windows(
    df_messages, time_windows, error_times
).sort_index()

# Render the filtered messages as collapsible HTML
render_following_error_reports(filtered_messages, expanded=False)